In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
from random import randint

In [ ]:
df = pd.read_csv(f'../player_info_scrapping/output/player_links.csv')
df = df.drop('Unnamed: 0', axis = 1)
df.index = range(0,len(df))
df = df.assign(Player URL = None, Date_of_Birth = None , City_of_Birth = None , Country_of_Birth = None , Height = None , Citizenship = None , Foot = None , Agent = None , Outfitter = None , Main_Position = None , Other_Position = None , Youth_Club = None)

In [ ]:
def get_data(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
    request = rq.get(url,headers=headers)
    soup = bs(request.text, 'html.parser')
    
    title_spans = soup.find_all('span')
    
    try:
        name = ' '.join([word for word in soup.find('h1').text.split() if not any(i.isdigit() for i in word)])
    except IndexError:
        name = None
    
    try:
        date_of_birth = [span.find_next('span').text for span in title_spans if 'Date of birth:' in span.text][0].strip()
    except IndexError:
        date_of_birth = None
        #change things for considering cities with more than 1 word name
    try:
        city_of_birth = [span.find_next('span').text for span in title_spans if 'Place of birth:' in span.text][0].split()[0]
    except IndexError:
        city_of_birth = None
        #change things for considering countries with more than 1 word name
    try:
        country_of_birth = [span.find_next('span').find('img')['title'] for span in title_spans if 'Place of birth:' in span.text][0]
    except IndexError:
        country_of_birth = None
        
    try:
        height = [span.find_next('span').text for span in title_spans if 'Height:' in span.text][0].split()[0]
    except IndexError:
        height = None
    #change things for considering countries with more than 1 word name
    try:
        citizenship = [span.find_next('span').text for span in title_spans if 'Citizenship:' in span.text][0].split()
    except AttributeError:
        citizenship = None
    
    try:
        foot = [span.find_next('span').text for span in title_spans if 'Foot:' in span.text][0]
    except IndexError:
        foot = None
    
    try:
        agent = [span.find_next('span').text for span in title_spans if 'Player agent:' in span.text][0].strip()
    except IndexError:
        agent = None
    
    try:
        outfitter = [span.find_next('span').text for span in title_spans if 'Outfitter:' in span.text][0]
    except IndexError:
        outfitter = None    
    
    try:
        main_position = soup.find('dt', text='Main position:').find_next('dd').text
    except AttributeError:
        main_position = None
    
    try:
        other_position = [dt.text for dt in soup.find('dt', text='Other position:').find_next_siblings('dd')]
    except AttributeError:
        other_position = None

    try:
        youth_club = soup.find('div', class_='box tm-player-additional-data viewport-tracking').find('div', class_='content').text.strip().split(',')
    except AttributeError:
        youth_club = None
        
    print(name)
    sleep(randint(1,10))
    
    return [name, date_of_birth, city_of_birth, country_of_birth, height, citizenship, foot, agent, outfitter, main_position, other_position, youth_club]

In [ ]:
df[['Name', 'Date_of_Birth', 'City_of_Birth', 'Country_of_Birth', 'Height', 'Citizenship', 'Foot', 'Agent', 'Outfitter', 'Main_Position', 'Other_Position', 'Youth_Club']] = df['URL'].apply(lambda url: pd.Series(get_data(url)))
df.to_csv('output/player_info.csv')